<a href="https://colab.research.google.com/github/MaJu502/proyectoDL/blob/main/ProyectoDL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install opencv-python

In [48]:
!pip install mtcnn
#https://medium.com/lcc-unison/aplicaci%C3%B3n-de-reconocimiento-facial-en-tiempo-real-para-identificar-a-alumnos-de-la-licenciatura-en-2e2b53870995

In [49]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from mtcnn import MTCNN
from torchvision.transforms import functional as F

In [50]:
import os
import cv2

# Carpeta que contiene los videos
video_folder = 'validation_videos/'

# Carpeta de salida para guardar los frames
output_folder = 'validation_frames/'

# Crea la carpeta de salida si no existe
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Lista de archivos en la carpeta de videos
video_files = os.listdir(video_folder)

for video_file in video_files:
    if video_file.endswith('.mp4') or video_file.endswith('.avi'):
        video_path = os.path.join(video_folder, video_file)

        # Abre el video
        cap = cv2.VideoCapture(video_path)

        # Verifica si el video se abrió correctamente
        if not cap.isOpened():
            print(f"No se pudo abrir el video: {video_path}")
            continue

        frame_count = 0
        frame_interval = 10  # Captura un frame cada 5 frames

        # Lee y guarda los frames como imágenes
        while True:
            ret, frame = cap.read()

            if not ret:
                break

            if frame_count % frame_interval == 0:
                # Guarda el frame como una imagen
                frame_filename = os.path.join(output_folder, f'{os.path.splitext(video_file)[0]}_frame_{frame_count:04d}.png')
                cv2.imwrite(frame_filename, frame)

            frame_count += 1

        # Cierra el video
        cap.release()

        print(f'Se han guardado {frame_count // frame_interval} frames del video {video_file} como imágenes.')

print('Proceso completo.')

Se han guardado 34 frames del video videoFeliz.mp4 como imágenes.
Proceso completo.


In [51]:
# Model YOLO
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
# Model MTCNN
face_detector = MTCNN()

# Carpeta de salida para guardar resultados
output_folder = 'persons/'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Carpeta que contiene los frames de validación
folder = 'validation_frames/'  # Cambiar el nombre de la carpeta de frames de validación
files = os.listdir(folder)

# Listas para métricas
true_positives = 0
false_positives = 0
false_negatives = 0

# Evaluar cada elemento dentro de los frames de validación
for img_name in files:
    img_path = os.path.join("validation_frames", img_name)  # Ruta de la carpeta de validación
    img = Image.open(img_path)
    results = model(img)

    # Obtener resultados de bounding boxes
    bboxes = results.pred[0]

    # Flag para seguimiento de detecciones
    detection_flag = False

    for i, bbox in enumerate(bboxes):
        # Ver si la clase es de tipo person
        if bbox[-1] == 0:
            x1, y1, x2, y2 = bbox[0:4].int().tolist()

            # Recortar imagen para obtener únicamente la caja
            cropped_img = img.crop((x1, y1, x2, y2))

            # Recortar caja para obtener únicamente las caras
            image_np = np.array(cropped_img)
            face_results = face_detector.detect_faces(image_np)

            if len(face_results) > 0:
                detection_flag = True
                face_bbox = face_results[0]['box']
                x, y, w, h = face_bbox
                face_image = cropped_img.crop((x, y, x + w, y + h))

                # Guardar imagen en carpeta persona
                output_path = os.path.join(output_folder, f"person_{os.path.splitext(img_name)[0]}_{i}.jpg")
                face_image.save(output_path)

    if detection_flag:
        true_positives += 1
    else:
        false_negatives += 1

# Calcular falsos positivos
false_positives = len(files) - true_positives

# Calcular métricas
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"True Positives: {true_positives}")
print(f"False Positives: {false_positives}")
print(f"False Negatives: {false_negatives}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-11-13 Python-3.10.12 torch-2.1.0+cu118 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


1/1 [==============================] - 0s 45ms/step
True Positives: 70
False Positives: 0
False Negatives: 0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [52]:
pip install joblib

In [53]:
import joblib

# Guardar el modelo YOLO
yolo_model_path = 'yolo_model.pkl'
joblib.dump(model, yolo_model_path)

# Guardar el modelo MTCNN (asegúrate de haber importado MTCNN correctamente)
mtcnn_model_path = 'mtcnn_model.pkl'
joblib.dump(face_detector, mtcnn_model_path)


['mtcnn_model.pkl']

In [54]:
# Cargar el modelo YOLO
loaded_yolo_model = joblib.load(yolo_model_path)

# Cargar el modelo MTCNN
loaded_mtcnn_model = joblib.load(mtcnn_model_path)


In [55]:
!zip -r /content/persons.zip /content/persons


  adding: content/persons/ (stored 0%)
  adding: content/persons/person_videoFeliz_frame_0045_0.jpg (deflated 2%)
  adding: content/persons/person_videoFeliz_frame_0130_0.jpg (deflated 2%)
  adding: content/persons/person_videoFeliz_frame_0210_0.jpg (deflated 2%)
  adding: content/persons/person_videoFeliz_frame_0300_0.jpg (deflated 2%)
  adding: content/persons/person_videoFeliz_frame_0085_0.jpg (deflated 2%)
  adding: content/persons/person_videoFeliz_frame_0200_0.jpg (deflated 2%)
  adding: content/persons/person_videoFeliz_frame_0225_0.jpg (deflated 2%)
  adding: content/persons/person_videoFeliz_frame_0145_0.jpg (deflated 2%)
  adding: content/persons/person_videoFeliz_frame_0190_0.jpg (deflated 2%)
  adding: content/persons/person_videoFeliz_frame_0175_0.jpg (deflated 2%)
  adding: content/persons/person_videoFeliz_frame_0330_0.jpg (deflated 2%)
  adding: content/persons/person_videoFeliz_frame_0235_0.jpg (deflated 2%)
  adding: content/persons/person_videoFeliz_frame_0000_0.jpg 

In [56]:
!zip -r /content/persons.zip /content/persons

updating: content/persons/ (stored 0%)
updating: content/persons/person_videoFeliz_frame_0045_0.jpg (deflated 2%)
updating: content/persons/person_videoFeliz_frame_0130_0.jpg (deflated 2%)
updating: content/persons/person_videoFeliz_frame_0210_0.jpg (deflated 2%)
updating: content/persons/person_videoFeliz_frame_0300_0.jpg (deflated 2%)
updating: content/persons/person_videoFeliz_frame_0085_0.jpg (deflated 2%)
updating: content/persons/person_videoFeliz_frame_0200_0.jpg (deflated 2%)
updating: content/persons/person_videoFeliz_frame_0225_0.jpg (deflated 2%)
updating: content/persons/person_videoFeliz_frame_0145_0.jpg (deflated 2%)
updating: content/persons/person_videoFeliz_frame_0190_0.jpg (deflated 2%)
updating: content/persons/person_videoFeliz_frame_0175_0.jpg (deflated 2%)
updating: content/persons/person_videoFeliz_frame_0330_0.jpg (deflated 2%)
updating: content/persons/person_videoFeliz_frame_0235_0.jpg (deflated 2%)
updating: content/persons/person_videoFeliz_frame_0000_0.jpg 

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
!mkdir -p '/content/drive/MyDrive/persons/'
!cp '/content/persons.zip' '/content/drive/MyDrive/persons/'

In [59]:
# !rm -r /content/drive/MyDrive/persons/

In [60]:
# import os

# directory_path = './persons/'
# files = os.listdir(directory_path)

# for file in files:
#       file_path = os.path.join(directory_path, file)
#       os.remove(file_path)
#       print(f"Deleted: {file_path}")